In [5]:
from training_utils import eval_model
from tqdm.notebook import tqdm
import joblib
import numpy as np
results = []
names = ['Cheb', 'DotGat', 'GAT_v2', 'GCN2Conv', 'GAT', 'PNAConv', 'Sage']

In [4]:
for model_name in tqdm(names):
    bert_scores, linear_regression_score_bert, svm_score_bert, model_scores, linear_regression_score_model, svm_score_model = eval_model(model_name)
    joblib.dump(value=bert_scores, filename=f"../results/scores/scores_bert.pickle")
    joblib.dump(value=model_scores, filename=f"../results/scores/scores_{model_name}.pickle")
    print(f'Модель: {np.mean(model_scores):.2f}±{np.std(model_scores):.2f}')
    print(f'Берт: {np.mean(bert_scores):.2f}±{np.std(bert_scores):.2f}')
    print(f'SVR BERT: {svm_score_bert:.2f}')
    print(f'SVR модель: {svm_score_model:.2f}')
    print(f'Линейная регрессия BERT: {linear_regression_score_bert:.2f}')
    print(f'Линейная регрессия модель {linear_regression_score_model:.2f}')
    results.append([bert_scores, linear_regression_score_bert, svm_score_bert, model_scores, linear_regression_score_model, svm_score_model])
    print('---')

  0%|          | 0/7 [00:00<?, ?it/s]

Модель: -0.84±1.23
Берт: 0.16±0.56
SVR BERT: -0.08
SVR модель: -1.12
Линейная регрессия BERT: 0.09
Линейная регрессия модель -0.03
---


Модель: -0.30±0.42
Берт: 0.13±0.48
SVR BERT: -0.08
SVR модель: -0.44
Линейная регрессия BERT: 0.09
Линейная регрессия модель 0.11
---


Модель: -0.11±0.35
Берт: 0.15±0.44
SVR BERT: -0.08
SVR модель: -0.44
Линейная регрессия BERT: 0.09
Линейная регрессия модель 0.12
---


Модель: -0.05±0.74
Берт: 0.14±0.46
SVR BERT: -0.08
SVR модель: -0.67
Линейная регрессия BERT: 0.09
Линейная регрессия модель 0.11
---


Модель: -0.36±0.49
Берт: 0.23±0.41
SVR BERT: -0.08
SVR модель: -0.63
Линейная регрессия BERT: 0.09
Линейная регрессия модель 0.12
---


Модель: -0.25±1.34
Берт: 0.23±0.38
SVR BERT: -0.08
SVR модель: -0.61
Линейная регрессия BERT: 0.09
Линейная регрессия модель 0.11
---


Модель: 0.06±0.42
Берт: 0.16±0.44
SVR BERT: -0.08
SVR модель: -0.45
Линейная регрессия BERT: 0.09
Линейная регрессия модель 0.06
---


In [9]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from training_utils import eval_model, fix_okved
from tqdm.notebook import tqdm
import joblib
import numpy as np

In [10]:
def clf_score(version_name, bert_use=True):
    okved_data = pd.read_csv('../data/okved2/okved_2014_w_sections.csv', index_col=0)
    sections = okved_data['section_id'].values

    embeddings_bert = np.load('../data/okved2/okved_embeddings.npy')
    # creating mapping for gos description
    embeddings_model = np.load(f'../results/embs/{version_name}.npy')

    X_bert = embeddings_bert
    X_bert = StandardScaler().fit_transform(X_bert)
    X_model = embeddings_model
    X_model = StandardScaler().fit_transform(X_model)
    y = sections
    model = LogisticRegression()

    bert_scores = []
    linear_regression_score_bert, svm_score_bert = 0, 0
    if bert_use:
        model = LogisticRegression().fit(X_bert, y)
        linear_regression_score_bert = model.score(X_bert, y)
        model = SVC().fit(X_bert, y)
        svm_score_bert = model.score(X_bert, y)
        for i in tqdm(range(100), leave=False):
            model = MLPClassifier().fit(X_bert, y)
            score = model.score(X_bert, y) 
            bert_scores.append(score)
    model_scores = []
    model = LogisticRegression().fit(X_model, y)
    linear_regression_score_model = model.score(X_model, y)
    model = SVC().fit(X_model, y)
    svm_score_model = model.score(X_model, y)
    for i in tqdm(range(100), leave=False):
        model = MLPClassifier().fit(X_model, y)
        score = model.score(X_model, y) 
        model_scores.append(score)
    return bert_scores, linear_regression_score_bert, svm_score_bert, model_scores, linear_regression_score_model, svm_score_model


In [4]:
import warnings
warnings.filterwarnings('ignore')

In [7]:
results = []
names = ['Cheb', 'DotGat', 'GAT_v2', 'GCN2Conv', 'GAT', 'PNAConv', 'Sage']
for model_name in tqdm(names):
    print(model_name)
    bert_scores, linear_regression_score_bert, svm_score_bert, model_scores, linear_regression_score_model, svm_score_model = clf_score(model_name)
    joblib.dump(value=bert_scores, filename=f"../results/scores/scores_bert.pickle")
    joblib.dump(value=model_scores, filename=f"../results/scores/scores_{model_name}.pickle")
    print(f'Модель: {np.mean(model_scores):.2f}±{np.std(model_scores):.2f}')
    print(f'Берт: {np.mean(bert_scores):.2f}±{np.std(bert_scores):.2f}')
    print(f'SVR BERT: {svm_score_bert:.2f}')
    print(f'SVR модель: {svm_score_model:.2f}')
    print(f'Линейная регрессия BERT: {linear_regression_score_bert:.2f}')
    print(f'Линейная регрессия модель {linear_regression_score_model:.2f}')
    results.append([bert_scores, linear_regression_score_bert, svm_score_bert, model_scores, linear_regression_score_model, svm_score_model])
    print('---')

  0%|          | 0/7 [00:00<?, ?it/s]

Cheb


  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

Модель: 0.99±0.00
Берт: 1.00±0.00
SVR BERT: 1.00
SVR модель: 0.92
Линейная регрессия BERT: 1.00
Линейная регрессия модель 0.97
---
DotGat


...

  0%|          | 0/100 [00:00<?, ?it/s]

Модель: 1.00±0.00
Берт: 1.00±0.00
SVR BERT: 1.00
SVR модель: 1.00
Линейная регрессия BERT: 1.00
Линейная регрессия модель 1.00
---
Sage


  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

Модель: 1.00±0.00
Берт: 1.00±0.00
SVR BERT: 1.00
SVR модель: 0.99
Линейная регрессия BERT: 1.00
Линейная регрессия модель 1.00
---


In [6]:
model_name = 'rgcn'

In [8]:
print(model_name)
bert_scores, linear_regression_score_bert, svm_score_bert, model_scores, linear_regression_score_model, svm_score_model = eval_model(model_name)
joblib.dump(value=bert_scores, filename=f"../results/scores/scores_bert.pickle")
joblib.dump(value=model_scores, filename=f"../results/scores/scores_{model_name}.pickle")
print(f'Модель: {np.mean(model_scores):.2f}±{np.std(model_scores):.2f}')
print(f'Берт: {np.mean(bert_scores):.2f}±{np.std(bert_scores):.2f}')
print(f'SVR BERT: {svm_score_bert:.2f}')
print(f'SVR модель: {svm_score_model:.2f}')
print(f'Линейная регрессия BERT: {linear_regression_score_bert:.2f}')
print(f'Линейная регрессия модель {linear_regression_score_model:.2f}')
results.append([bert_scores, linear_regression_score_bert, svm_score_bert, model_scores, linear_regression_score_model, svm_score_model])
print('---')

rgcn


Модель: 0.17±0.44
Берт: 0.15±0.43
SVR BERT: -0.08
SVR модель: -0.54
Линейная регрессия BERT: 0.09
Линейная регрессия модель 0.06
---


In [11]:
print(model_name)
bert_scores, linear_regression_score_bert, svm_score_bert, model_scores, linear_regression_score_model, svm_score_model = clf_score(model_name)
print(f'Модель: {np.mean(model_scores):.2f}±{np.std(model_scores):.2f}')
print(f'Берт: {np.mean(bert_scores):.2f}±{np.std(bert_scores):.2f}')
print(f'SVR BERT: {svm_score_bert:.2f}')
print(f'SVR модель: {svm_score_model:.2f}')
print(f'Линейная регрессия BERT: {linear_regression_score_bert:.2f}')
print(f'Линейная регрессия модель {linear_regression_score_model:.2f}')
results.append([bert_scores, linear_regression_score_bert, svm_score_bert, model_scores, linear_regression_score_model, svm_score_model])
print('---')

rgcn


  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

Модель: 1.00±0.00
Берт: 1.00±0.00
SVR BERT: 1.00
SVR модель: 0.97
Линейная регрессия BERT: 1.00
Линейная регрессия модель 1.00
---


In [12]:
model_name='rgcn_'

In [13]:
print(model_name)
bert_scores, linear_regression_score_bert, svm_score_bert, model_scores, linear_regression_score_model, svm_score_model = eval_model(model_name)
joblib.dump(value=bert_scores, filename=f"../results/scores/scores_bert.pickle")
joblib.dump(value=model_scores, filename=f"../results/scores/scores_{model_name}.pickle")
print(f'Модель: {np.mean(model_scores):.2f}±{np.std(model_scores):.2f}')
print(f'Берт: {np.mean(bert_scores):.2f}±{np.std(bert_scores):.2f}')
print(f'SVR BERT: {svm_score_bert:.2f}')
print(f'SVR модель: {svm_score_model:.2f}')
print(f'Линейная регрессия BERT: {linear_regression_score_bert:.2f}')
print(f'Линейная регрессия модель {linear_regression_score_model:.2f}')
results.append([bert_scores, linear_regression_score_bert, svm_score_bert, model_scores, linear_regression_score_model, svm_score_model])
print('---')
print(model_name)
bert_scores, linear_regression_score_bert, svm_score_bert, model_scores, linear_regression_score_model, svm_score_model = clf_score(model_name)
print(f'Модель: {np.mean(model_scores):.2f}±{np.std(model_scores):.2f}')
print(f'Берт: {np.mean(bert_scores):.2f}±{np.std(bert_scores):.2f}')
print(f'SVR BERT: {svm_score_bert:.2f}')
print(f'SVR модель: {svm_score_model:.2f}')
print(f'Линейная регрессия BERT: {linear_regression_score_bert:.2f}')
print(f'Линейная регрессия модель {linear_regression_score_model:.2f}')
results.append([bert_scores, linear_regression_score_bert, svm_score_bert, model_scores, linear_regression_score_model, svm_score_model])
print('---')

rgcn_


Модель: 0.19±0.47
Берт: 0.14±0.51
SVR BERT: -0.08
SVR модель: -0.54
Линейная регрессия BERT: 0.09
Линейная регрессия модель 0.06
---
rgcn_


  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/100 [00:00<?, ?it/s]

Модель: 1.00±0.00
Берт: 1.00±0.00
SVR BERT: 1.00
SVR модель: 0.97
Линейная регрессия BERT: 1.00
Линейная регрессия модель 1.00
---
